# **Elo Merchant Category Recommendation**



---



# Table of Contents


>*   [8. Improved Solution](#section8)
  *   [8.1. Splitting Features and Labels](#section8.1)
  *   [8.2. LGB Model with KFold](#section8.2)
      *   [8.2.1. Hyperparameter Tuning](#section8.2.1)
      *   [8.2.2. Model Building](#section8.2.2)
  *   [8.3. LGB Model with Repeated KFold](#section8.3)
      *   [8.3.1. Hyperparameter Tuning](#section8.3.1)
      *   [8.3.2. Model Building](#section8.3.2)
  *   [8.4. Summary](#section8.4)
  *   [8.5. Final Submission](#section8.5)








---



# Importing Libraries

In [ ]:
# GPU setup for LGBM
! git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 22620, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 22620 (delta 127), reused 165 (delta 95), pack-reused 22385
Receiving objects: 100% (22620/22620), 17.60 MiB | 29.22 MiB/s, done.
Resolving deltas: 100% (16566/16566), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), done.        
remote: Com

In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--

In [ ]:
!pip install optuna

     |████████████████████████████████| 307kB 29.6MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 174kB 43.3MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 112kB 41.6MB/s 
     |████████████████████████████████| 143kB 46.9MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=24fe5bfd21e5178b85039412b9dbf341ab1a1d11a931ac6e3d0118c42a559262
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [ ]:
# importing the required libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedKFold, GridSearchCV, RandomizedSearchCV
import lightgbm as lgb
from prettytable import PrettyTable
import gc
import pickle
import optuna
import warnings
warnings.filterwarnings("ignore")

# Loading Datasets

In [ ]:
#ref: https://www.kaggle.com/fabiendaniel/elo-world
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Case Study 1/Data/Dataset/'
model_path = '/content/drive/MyDrive/Colab Notebooks/Case Study 1/Data/Model/'
result_path = '/content/drive/MyDrive/Colab Notebooks/Case Study 1/Data/Result/'

In [ ]:
train = pd.read_csv(dataset_path + 'train_FE2.csv')
test = pd.read_csv(dataset_path + 'test_FE2.csv')

print('Shape of train data =', train.shape)
print('Shape of test data =', test.shape)

Shape of train data = (201917, 168)
Shape of test data = (123623, 166)


In [ ]:
# reduce the memory of all datasets
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 78.95 Mb (69.5% reduction)
Mem. usage decreased to 47.75 Mb (69.5% reduction)


<a name='section8'><a/>
# 8. Improved Solution

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month,year,dayofweek,weekofyear,elapsed_time,old_purchase_amount_sum,old_purchase_amount_max,old_purchase_amount_min,old_purchase_amount_mean,old_purchase_amount_var,old_purchase_amount_skew,old_installments_sum,old_installments_max,old_installments_mean,old_installments_var,old_installments_skew,old_purchase_date_max,old_purchase_date_min,old_month_lag_max,old_month_lag_min,old_month_lag_mean,old_month_lag_var,old_month_lag_skew,old_month_diff_max,old_month_diff_min,old_month_diff_mean,old_month_diff_var,old_month_diff_skew,old_weekend_sum,old_weekend_mean,old_weekday_sum,old_weekday_mean,old_authorized_flag_sum,old_authorized_flag_mean,...,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_dayofweek_nunique,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_price_sum,new_price_mean,new_price_max,new_price_min,new_price_var,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,old_first_buy,new_first_buy,card_id_total,purchase_amount_total,outliers
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,6,2017,3,22,1487,-165.968735,2.258394,-0.739258,-0.638341,0.045003,10.242188,4,1,0.015381,0.015206,7.921875,1.519551e+09,1.498573e+09,0,-8,-3.912109,5.750000,0.065979,41,40,40.53125,0.250000,-0.124023,90,0.346191,835,3.210938,247.0,0.950195,...,12.867188,8.0,16.0,7.0,13.304688,10.0,17.0,7.0,17.0,16.437500,5.0,31.0,10.0,23.0,14.0,-inf,-inf,-inf,-inf,NaN,-23.296875,-29.515625,-11.843750,30.085054,0.887695,-0.014229,-0.018112,-0.007221,1.138449e-05,0.895508,-0.550293,-0.592773,54.0,2.347656,1154.0,26,277.0,283.0,-179.210922,0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,2017,6,52,1638,-210.006332,4.630300,-0.742188,-0.600018,0.148200,8.812500,543,10,1.551758,2.282448,3.718750,1.517438e+09,1.483720e+09,0,-12,-5.031250,14.476562,-0.258057,42,41,41.46875,0.248657,0.184448,132,0.377197,1177,3.363281,339.0,0.968750,...,11.164062,6.0,17.0,4.0,9.000000,5.0,13.0,4.0,4.0,13.500000,1.0,30.0,4.0,6.0,5.0,-4.355469,-0.725911,-0.701660,-0.739258,0.000207,-30.125000,-31.046875,-28.765625,0.833826,0.681641,-0.017487,-0.017899,-0.017120,5.960464e-08,-0.262451,-0.550293,-0.606445,56.0,9.335938,1185.0,5,396.0,356.0,-214.361801,0
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,8,2016,0,31,1791,-29.167391,-0.145847,-0.729980,-0.678311,0.007635,5.625000,0,0,0.000000,0.000000,0.000000,1.519758e+09,1.484123e+09,0,-13,-8.601562,14.765625,0.725586,41,40,40.31250,0.224854,0.771484,11,0.255859,142,3.302734,41.0,0.953613,...,17.000000,17.0,17.0,1.0,17.000000,17.0,17.0,1.0,1.0,28.000000,28.0,28.0,1.0,1.0,1.0,-inf,-inf,-inf,-inf,NaN,-28.000000,-28.000000,-28.000000,NaN,NaN,-0.017502,-0.017502,-0.017502,NaN,NaN,-0.548828,-0.592773,0.0,0.000000,1155.0,163,635.0,44.0,-29.867586,0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,9,2017,4,35,1395,-49.491364,1.445596,-0.740723,-0.642745,0.068447,6.968750,84,3,1.090820,0.346890,0.776855,1.519818e+09,1.506443e+09,0,-5,-2.832031,3.248047,0.557617,41,40,40.37500,0.240967,0.461670,11,0.142822,215,2.792969,77.0,1.000000,...,13.000000,7.0,21.0,5.0,13.859375,10.0,16.0,4.0,7.0,13.140625,4.0,24.0,5.0,7.0,6.0,-3.520996,-0.502999,0.566895,-0.734375,0.225098,-26.906250,-30.109375,-22.671875,8.349981,0.736816,-0.016434,-0.017914,-0.014175,2.264977e-06,0.789062,-0.556641,-0.604492,41.0,5.855469,1165.0,25,187.0,84.0,-54.147614,0
4,2017-11-01,C_ID_cdbd2c0db2,0.008057,0.014168,0.010284,-0.159790,11,2017,2,44,1334,-48.687656,7.193041,-0.746094,-0.366073,1.828

In [ ]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,month,year,dayofweek,weekofyear,elapsed_time,old_purchase_amount_sum,old_purchase_amount_max,old_purchase_amount_min,old_purchase_amount_mean,old_purchase_amount_var,old_purchase_amount_skew,old_installments_sum,old_installments_max,old_installments_mean,old_installments_var,old_installments_skew,old_purchase_date_max,old_purchase_date_min,old_month_lag_max,old_month_lag_min,old_month_lag_mean,old_month_lag_var,old_month_lag_skew,old_month_diff_max,old_month_diff_min,old_month_diff_mean,old_month_diff_var,old_month_diff_skew,old_weekend_sum,old_weekend_mean,old_weekday_sum,old_weekday_mean,old_authorized_flag_sum,old_authorized_flag_mean,old_category_1_sum,...,new_hour_nunique,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_dayofweek_nunique,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_price_sum,new_price_mean,new_price_max,new_price_min,new_price_var,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,old_first_buy,new_first_buy,card_id_total,purchase_amount_total
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,4,2017,5,13,1548,-40.733734,0.235676,-0.744141,-0.599121,0.036967,2.335938,141,12,2.074219,4.248244,2.458984,1.514510e+09,1.491330e+09,0,-8,-3.632812,6.027344,-0.116150,43,42,42.50000,0.253662,0.000000,12,0.176514,190,2.794922,44.0,0.646973,23.0,...,3.0,13.664062,9.0,19.0,3.0,7.000000,5.0,9.0,3.0,3.0,15.664062,3.0,28.0,3.0,3.0,3.0,-1.521810,-0.507270,-0.127767,-0.722168,0.108643,-25.015625,-30.328125,-16.484375,55.668808,1.577148,-0.014038,-0.017197,-0.008911,0.000020,1.593750,-0.550293,-0.391602,25.0,8.335938,1214.0,3,308.0,71.0,-42.511078
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,2017,6,52,1638,-49.136513,0.318817,-0.731934,-0.629883,0.024025,3.771484,83,4,1.064453,0.164669,6.492188,1.518989e+09,1.484322e+09,0,-13,-10.414062,4.687500,2.716797,41,40,40.50000,0.253174,-0.052307,17,0.217896,229,2.935547,77.0,0.987305,2.0,...,8.0,15.218750,9.0,23.0,6.0,12.445312,9.0,16.0,6.0,7.0,11.335938,3.0,20.0,6.0,9.0,8.0,-5.606771,-0.622974,-0.168783,-0.740723,0.030731,-26.953125,-30.375000,-20.250000,9.400031,1.306641,-0.016190,-0.018066,-0.012657,0.000003,1.230469,-0.556152,-0.534668,48.0,5.332031,1163.0,12,426.0,87.0,-55.081825
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,8,2017,1,31,1426,4.528841,2.525867,-0.536621,0.348389,0.821827,1.332031,44,10,3.384766,13.589744,0.641113,1.517598e+09,1.503673e+09,0,-6,-2.076172,3.076172,-0.738281,41,40,40.84375,0.140991,-2.179688,0,0.000000,34,2.615234,9.0,0.692383,1.0,...,2.0,13.500000,13.0,14.0,2.0,10.000000,9.0,11.0,2.0,2.0,7.000000,1.0,13.0,2.0,2.0,2.0,-0.634180,-0.317090,0.090430,-0.724609,0.332031,3.683594,-29.703125,37.062500,2230.128662,NaN,0.002192,-0.017670,0.022049,0.000789,NaN,-0.549805,-0.284424,11.0,5.500000,1201.0,24,212.0,15.0,4.708528
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,12,2017,4,48,1304,-13.690715,0.087965,-0.731934,-0.526367,0.048032,1.864258,38,6,1.461914,2.178462,1.537109,1.519127e+09,1.512392e+09,0,-2,-1.230469,0.904785,0.502441,41,40,40.65625,0.235352,-0.687012,7,0.269287,64,2.460938,26.0,1.000000,0.0,...,7.0,18.203125,11.0,21.0,7.0,12.101562,9.0,16.0,5.0,8.0,13.601562,4.0,31.0,8.0,10.0,10.0,-3.805725,-0.380572,-0.037333,-0.671875,0.058167,-23.312500,-26.875000,-18.375000,8.692524,0.425049,-0.014153,-0.016800,-0.010925,0.000003,0.306641,-0.550293,-0.284424,44.0,4.398438,1166.0,3,93.0,36.0,-19.436810
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011383,0.011429,12,2015,1,49,2035,25.139385,15.782255,-0.7465

<a name='section8.1'><a/>
## 8.1. Splitting features and labels

In [ ]:
train_data_y = train['target'].values
train_data_x = train.drop(['card_id','first_active_month','target','outliers'], axis=1)

test_data_x = test.drop(['card_id','first_active_month'], axis=1)

print('Shape of final train data :', train_data_x.shape, train_data_y.shape)
print('Shape of final test data :', test_data_x.shape)

Shape of final train data : (201917, 164) (201917,)
Shape of final test data : (123623, 164)


<a name='section8.2'><a/>
## 8.2. LGB Model with KFold

<a name='section8.2.1'><a/>
### 8.2.1. Hyperparameter Tuning

In [ ]:
# ref: https://colab.research.google.com/drive/1I8abciKFXBxkoXrcwLiG7_BrYskLnV8k#scrollTo=8lOM3b6iO3it

OPTUNE_EARLY_STOPING = 10

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    early_stop = OPTUNE_EARLY_STOPING
    early_stop_count = 0
    best_score = None

def early_stopping_opt(study, trial):
    if EarlyStoppingExceeded.best_score == None:
      EarlyStoppingExceeded.best_score = study.best_value

    if study.best_value < EarlyStoppingExceeded.best_score:
        EarlyStoppingExceeded.best_score = study.best_value
        EarlyStoppingExceeded.early_stop_count = 0
    else:
      if EarlyStoppingExceeded.early_stop_count > EarlyStoppingExceeded.early_stop:
            EarlyStoppingExceeded.early_stop_count = 0
            best_score = None
            raise EarlyStoppingExceeded()
      else:
            EarlyStoppingExceeded.early_stop_count=EarlyStoppingExceeded.early_stop_count+1
    print(f'EarlyStop counter: {EarlyStoppingExceeded.early_stop_count}, Best score: {study.best_value} and {EarlyStoppingExceeded.best_score}')
    return

In [ ]:
# ref: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
def objective(trial):
    params = {
              'objective': 'regression', 
              'metric': 'rmse',
              'verbose': -1,  
              "learning_rate": 0.01,
              'device': 'gpu',  
              'seed': 326,
              'boosting_type': 'gbdt', 
              'n_jobs': -1,
              'num_leaves': trial.suggest_int('num_leaves', 16, 64), 
              'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.001, 1),
              'subsample': trial.suggest_uniform('subsample', 0.001, 1), 
              'max_depth': trial.suggest_int('max_depth', 1, 12),
              'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 10), 
              'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 10),
              'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 10), 
              'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 45),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 16, 64)
              }

    lgb_model = lgb.Dataset(train_data_x, train_data_y, free_raw_data=False)
    folds = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_results = lgb.cv(params=params, train_set=lgb_model,
                      metrics=['rmse'], nfold=3,
                      folds=folds.split(train_data_x, train_data_y),
                      num_boost_round=10000,
                      early_stopping_rounds=200, 
                      verbose_eval=False, seed=47)
    gc.collect()
    return cv_results['rmse-mean'][-1]

In [ ]:
study = optuna.create_study()
try:
    study.optimize(objective, n_trials=100, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNE_EARLY_STOPING}')

[I 2021-06-28 04:35:43,205] A new study created in memory with name: no-name-a1590e3a-f47c-4d51-8477-4ea9b60e7f92
[I 2021-06-28 04:40:10,721] Trial 0 finished with value: 3.68424549852479 and parameters: {'num_leaves': 58, 'colsample_bytree': 0.020893146132064045, 'subsample': 0.8978559677653196, 'max_depth': 4, 'reg_alpha': 3.0956912814862356, 'reg_lambda': 6.256436796481032, 'min_split_gain': 6.43230231390557, 'min_child_weight': 23.189844569251658, 'min_data_in_leaf': 42}. Best is trial 0 with value: 3.68424549852479.


EarlyStop counter: 1, Best score: 3.68424549852479 and 3.68424549852479


[I 2021-06-28 04:42:46,321] Trial 1 finished with value: 3.7306916633590994 and parameters: {'num_leaves': 61, 'colsample_bytree': 0.6020530588836303, 'subsample': 0.8388376191891913, 'max_depth': 1, 'reg_alpha': 4.420416916373067, 'reg_lambda': 4.344015427026745, 'min_split_gain': 1.0616044060204943, 'min_child_weight': 35.96994264366056, 'min_data_in_leaf': 47}. Best is trial 0 with value: 3.68424549852479.


EarlyStop counter: 2, Best score: 3.68424549852479 and 3.68424549852479


[I 2021-06-28 04:44:20,450] Trial 2 finished with value: 3.6550199584805583 and parameters: {'num_leaves': 63, 'colsample_bytree': 0.19573971822927025, 'subsample': 0.7088324424847781, 'max_depth': 5, 'reg_alpha': 5.4086079425810185, 'reg_lambda': 6.033612964635145, 'min_split_gain': 2.845508478099261, 'min_child_weight': 43.93998926326262, 'min_data_in_leaf': 62}. Best is trial 2 with value: 3.6550199584805583.


...

EarlyStop counter: 10, Best score: 3.6520213143512392 and 3.6520213143512392


[I 2021-06-28 05:32:49,305] Trial 30 finished with value: 3.662093680981066 and parameters: {'num_leaves': 46, 'colsample_bytree': 0.06447034419985942, 'subsample': 0.1554306488212582, 'max_depth': 11, 'reg_alpha': 5.226806906717927, 'reg_lambda': 5.691853783743792, 'min_split_gain': 4.733266471763692, 'min_child_weight': 0.5723659186182193, 'min_data_in_leaf': 44}. Best is trial 19 with value: 3.6520213143512392.


EarlyStop counter: 11, Best score: 3.6520213143512392 and 3.6520213143512392


[I 2021-06-28 05:34:35,550] Trial 31 finished with value: 3.6537800661417137 and parameters: {'num_leaves': 21, 'colsample_bytree': 0.35875826729420746, 'subsample': 0.33975575414581555, 'max_depth': 9, 'reg_alpha': 7.444960535281579, 'reg_lambda': 2.347240364007241, 'min_split_gain': 6.785054407812734, 'min_child_weight': 9.996180080485829, 'min_data_in_leaf': 35}. Best is trial 19 with value: 3.6520213143512392.


EarlyStopping Exceeded: No new best scores on iters 10


In [ ]:
best_trial = study.best_trial

print('Best Trial Value: {}'.format(best_trial.value))
print('='*50)
print('Best Trial Params:')
for k, v in best_trial.params.items():
    print('{}: {}'.format(k, v))

Best Trial Value: 3.6520213143512392
Best Trial Params:
num_leaves: 27
colsample_bytree: 0.4224442272238475
subsample: 0.27679399577774655
max_depth: 12
reg_alpha: 5.865290338680648
reg_lambda: 3.037629214884319
min_split_gain: 7.369831245684944
min_child_weight: 20.391715687283007
min_data_in_leaf: 31


In [ ]:
optuna.visualization.plot_optimization_history(study)

<a name='section8.2.2'><a/>
### 8.2.2. Model Building

In [ ]:
params = {
          'objective': 'regression',
          'boosting_type': 'gbdt',
          'metric': 'rmse',
          'learning_rate': 0.01,
          'num_leaves': 27,
          'colsample_bytree': 0.4224442272238475,
          'subsample': 0.27679399577774655,
          'max_depth': 12,
          'reg_alpha': 5.865290338680648,
          'reg_lambda': 3.037629214884319,
          'min_split_gain': 7.369831245684944,
          'min_child_weight': 20.391715687283007,
          'min_data_in_leaf': 31,
          'verbosity': -1
          }

test_data_y = 0
kfold = KFold(n_splits=5, random_state=2018, shuffle=True)

for f, (train_idx, val_idx) in enumerate(kfold.split(train_data_x, train_data_y)):
    print('='*80)
    print('FOLD :', f)
    x_train_, x_val_, y_train_, y_val_ = train_data_x.loc[train_idx,:], train_data_x.loc[val_idx,:], train_data_y[train_idx], train_data_y[val_idx]

    lgtrain = lgb.Dataset(x_train_, label=y_train_)
    lgval = lgb.Dataset(x_val_, label=y_val_)

    evals_result = {}
    model6 = lgb.train(params, lgtrain, 10000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100, evals_result=evals_result)
    test_data_y += model6.predict(test_data_x, num_iteration=model6.best_iteration)

test_data_y /= 5.

FOLD : 0
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.67688	valid_1's rmse: 3.73191
[200]	training's rmse: 3.60421	valid_1's rmse: 3.70197
[300]	training's rmse: 3.56018	valid_1's rmse: 3.6899
[400]	training's rmse: 3.52527	valid_1's rmse: 3.68321
[500]	training's rmse: 3.49784	valid_1's rmse: 3.67968
[600]	training's rmse: 3.47407	valid_1's rmse: 3.67719
[700]	training's rmse: 3.45279	valid_1's rmse: 3.67545
[800]	training's rmse: 3.43314	valid_1's rmse: 3.67447
[900]	training's rmse: 3.41462	valid_1's rmse: 3.67396
[1000]	training's rmse: 3.39872	valid_1's rmse: 3.67361
[1100]	training's rmse: 3.384	valid_1's rmse: 3.67336
[1200]	training's rmse: 3.37006	valid_1's rmse: 3.67296
[1300]	training's rmse: 3.35625	valid_1's rmse: 3.67294
[1400]	training's rmse: 3.34314	valid_1's rmse: 3.67247
[1500]	training's rmse: 3.33013	valid_1's rmse: 3.67244
[1600]	training's rmse: 3.31756	valid_1's rmse: 3.67263
Early stopping, best iteration is:
[1512]	tra

In [ ]:
df = pd.DataFrame({'card_id': test['card_id'].values})
df['target'] = test_data_y
df.to_csv(result_path + 'lgb_kfold_test.csv', index=False)

In [ ]:
filename = model_path + 'lgb_kfold_model.sav'
pickle.dump(model6, open(filename, 'wb'))

<a name='section8.3'><a/>
## 8.3. LGB Model with Repeated KFold

<a name='section8.3.1'><a/>
### 8.3.1. Hyperparameter Tuning

In [ ]:
# ref: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
def objective(trial):
    params = {
              'objective': 'regression', 
              'metric': 'rmse',
              'verbose': -1,  
              "learning_rate": 0.01,
              'device': 'gpu',  
              'seed': 326,
              'boosting_type': 'gbdt', 
              'n_jobs': -1,
              'num_leaves': trial.suggest_int('num_leaves', 16, 64), 
              'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.001, 1),
              'subsample': trial.suggest_uniform('subsample', 0.001, 1), 
              'max_depth': trial.suggest_int('max_depth', 1, 12),
              'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 10), 
              'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 10),
              'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 10), 
              'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 45),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 16, 64)
              }

    lgb_model = lgb.Dataset(train_data_x, train_data_y, free_raw_data=False)
    folds = RepeatedKFold(n_splits=3, n_repeats=2, random_state=4950)
    cv_results = lgb.cv(params=params, train_set=lgb_model,
                      metrics=['rmse'], nfold=3,
                      folds=folds.split(train_data_x, train_data_y),
                      num_boost_round=10000,
                      early_stopping_rounds=200, 
                      verbose_eval=False, seed=47)
    gc.collect()
    return cv_results['rmse-mean'][-1]

In [ ]:
study = optuna.create_study()
try:
    study.optimize(objective, n_trials=100, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNE_EARLY_STOPING}')

[I 2021-06-28 06:03:23,158] A new study created in memory with name: no-name-75fa9375-f84b-4dbf-b4c5-3832adb41059
[I 2021-06-28 06:06:40,637] Trial 0 finished with value: 3.6555665503673236 and parameters: {'num_leaves': 21, 'colsample_bytree': 0.21300277856242064, 'subsample': 0.8307771535128304, 'max_depth': 9, 'reg_alpha': 2.697032583560982, 'reg_lambda': 5.698188424718975, 'min_split_gain': 0.8376115467799072, 'min_child_weight': 39.00326490091637, 'min_data_in_leaf': 26}. Best is trial 0 with value: 3.6555665503673236.


EarlyStop counter: 1, Best score: 3.6555665503673236 and 3.6520213143512392


[I 2021-06-28 06:10:16,092] Trial 1 finished with value: 3.654954535613902 and parameters: {'num_leaves': 21, 'colsample_bytree': 0.5938685060380257, 'subsample': 0.6533114093552098, 'max_depth': 11, 'reg_alpha': 5.182549408170342, 'reg_lambda': 4.517649991671185, 'min_split_gain': 9.166131454824322, 'min_child_weight': 0.3502883463325207, 'min_data_in_leaf': 60}. Best is trial 1 with value: 3.654954535613902.


EarlyStop counter: 2, Best score: 3.654954535613902 and 3.6520213143512392


[I 2021-06-28 06:13:54,234] Trial 2 finished with value: 3.6585421347792604 and parameters: {'num_leaves': 54, 'colsample_bytree': 0.13829630803996099, 'subsample': 0.44825843818479644, 'max_depth': 4, 'reg_alpha': 5.195528685367869, 'reg_lambda': 0.9722731093656012, 'min_split_gain': 4.9442283033450805, 'min_child_weight': 14.92067215523907, 'min_data_in_leaf': 45}. Best is trial 1 with value: 3.654954535613902.


...

EarlyStop counter: 10, Best score: 3.6546437009884354 and 3.6520213143512392


[I 2021-06-28 06:42:12,835] Trial 10 finished with value: 3.731266331757715 and parameters: {'num_leaves': 45, 'colsample_bytree': 0.9527922837314363, 'subsample': 0.5048016573601142, 'max_depth': 1, 'reg_alpha': 9.79725334760677, 'reg_lambda': 7.433218896068545, 'min_split_gain': 6.83747552587654, 'min_child_weight': 2.4852615156717803, 'min_data_in_leaf': 33}. Best is trial 6 with value: 3.6546437009884354.


EarlyStop counter: 11, Best score: 3.6546437009884354 and 3.6520213143512392


[I 2021-06-28 06:45:33,428] Trial 11 finished with value: 3.6563736759736614 and parameters: {'num_leaves': 16, 'colsample_bytree': 0.6618760442081402, 'subsample': 0.5813110424269233, 'max_depth': 6, 'reg_alpha': 3.11202514452214, 'reg_lambda': 6.647267395351964, 'min_split_gain': 9.639302441543808, 'min_child_weight': 1.6993472925186062, 'min_data_in_leaf': 64}. Best is trial 6 with value: 3.6546437009884354.


EarlyStopping Exceeded: No new best scores on iters 10


In [ ]:
best_trial = study.best_trial

print('Best Trial Value: {}'.format(best_trial.value))
print('='*50)
print('Best Trial Params:')
for k, v in best_trial.params.items():
    print('{}: {}'.format(k, v))

Best Trial Value: 3.6546437009884354
Best Trial Params:
num_leaves: 39
colsample_bytree: 0.6263827102570619
subsample: 0.45932933016472877
max_depth: 6
reg_alpha: 4.967454444016626
reg_lambda: 6.38090341223462
min_split_gain: 3.3875132923280904
min_child_weight: 1.7839422937348552
min_data_in_leaf: 44


In [ ]:
optuna.visualization.plot_optimization_history(study)

<a name='section8.3.2'><a/>
### 8.3.2. Model Building

In [ ]:
params = {
          'objective': 'regression',
          'boosting_type': 'gbdt',
          'metric': 'rmse',
          'learning_rate': 0.01,
          'num_leaves': 39,
          'colsample_bytree': 0.6263827102570619,
          'subsample': 0.45932933016472877,
          'max_depth': 6,
          'reg_alpha': 4.967454444016626,
          'reg_lambda': 6.38090341223462,
          'min_split_gain': 3.3875132923280904,
          'min_child_weight': 1.7839422937348552,
          'min_data_in_leaf': 44,
          'verbosity': -1
          }

test_data_y = 0
kfold = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4950)

for f, (train_idx, val_idx) in enumerate(kfold.split(train_data_x, train_data_y)):
  print('='*80)
  print('FOLD :', f)
  x_train_, x_val_, y_train_, y_val_ = train_data_x.loc[train_idx,:], train_data_x.loc[val_idx,:], train_data_y[train_idx], train_data_y[val_idx]

  lgtrain = lgb.Dataset(x_train_, label=y_train_)
  lgval = lgb.Dataset(x_val_, label=y_val_)

  evals_result = {}
  model_r = lgb.train(params, lgtrain, 1000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100, evals_result=evals_result)
  test_data_y += model_r.predict(test_data_x, num_iteration=model_r.best_iteration)

test_data_y /= 5.

FOLD : 0
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.6809	valid_1's rmse: 3.71846
[200]	training's rmse: 3.61827	valid_1's rmse: 3.68735
[300]	training's rmse: 3.58191	valid_1's rmse: 3.67755
[400]	training's rmse: 3.55708	valid_1's rmse: 3.67347
[500]	training's rmse: 3.53764	valid_1's rmse: 3.6705
[600]	training's rmse: 3.51937	valid_1's rmse: 3.66832
[700]	training's rmse: 3.50459	valid_1's rmse: 3.66769
[800]	training's rmse: 3.49046	valid_1's rmse: 3.66737
Early stopping, best iteration is:
[757]	training's rmse: 3.49659	valid_1's rmse: 3.6671
FOLD : 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.64285	valid_1's rmse: 3.86853
[200]	training's rmse: 3.58253	valid_1's rmse: 3.83386
[300]	training's rmse: 3.54702	valid_1's rmse: 3.82052
[400]	training's rmse: 3.52341	valid_1's rmse: 3.81402
[500]	training's rmse: 3.50405	valid_1's rmse: 3.80982
[600]	training's rmse: 3.48834	valid_1's rmse: 3.80701
[

In [ ]:
df = pd.DataFrame({'card_id': test['card_id'].values})
df['target'] = test_data_y
df.to_csv(result_path + 'lgb_rkfold_test.csv', index=False)

<a name='section8.4'><a/>
## 8.4. Summary

In [ ]:
x = PrettyTable(['Model', 'Train RMSE', 'Val RMSE'])
x.add_row(['LGB with KFold', 3.28632, 3.70554])
x.add_row(['LGB with Repeated KFold', 3.48606, 3.5976])

print(x)

+-------------------------+------------+----------+
|          Model          | Train RMSE | Val RMSE |
+-------------------------+------------+----------+
|      LGB with KFold     |  3.28632   | 3.70554  |
| LGB with Repeated KFold |  3.48606   |  3.5976  |
+-------------------------+------------+----------+


<a name='section8.5'><a/>
## 8.5. Final Submission

Kaggle Leader Board Score:

![kaggle-score-baseline](https://drive.google.com/uc?export=view&id=1K0qFpF-lLOLqj7oNixlyc8CCSCmuK_TK)




---



# References:

*   https://www.kaggle.com/c/elo-merchant-category-recommendation
*   https://stackoverflow.com/
*   https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

